# Atividade 1 - Problema MAX-QBF com Set Cover

#### MO824 - Tópicos em Otimização Combinatória - Unicamp

#### Alunos:

Carlos E. Cavalieri Furtado -
RA 219748 -
c219748@dac.unicamp.br

Samuel Batista De Souza -
RA 205681 -
s205681@dac.unicamp.br

Este relatório apresenta a modelagem e resolução computacional do problema de maximização de função binária quadrática com restrições de cobertura de conjuntos (MAX-SC-QBF), desenvolvido para a disciplina MO824. O trabalho consiste na formulação matemática através de programação linear inteira, implementação de um gerador de instâncias e análise experimental utilizando o solver Gurobi para resolver 15 instâncias combinando cinco tamanhos (n ∈ {25, 50, 100, 200, 400}) e três padrões de geração de conjuntos, com limite de 10 minutos por instância.

#### 1. Configurações iniciais

##### 1.1 Importação de bibliotecas

In [1]:
# import libs
import numpy as np
import pandas as pd
import time
import os
from itertools import combinations
import random

In [2]:
# try to import Gurobi API or install it
try:
  import gurobipy as gp
  from gurobipy import GRB
except:
  !pip install gurobipy
  import gurobipy as gp
  from gurobipy import GRB

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 8.6 MB/s eta 0:00:00 0:00:01


##### 1.2 Instalação da licensa do Gurobi

In [3]:
# Gurobi WLS License
WLSACCESSID=""
WLSSECRET=""
LICENSEID=

params = {
"WLSACCESSID": WLSACCESSID,
"WLSSECRET": WLSSECRET,
"LICENSEID": LICENSEID
}
env = gp.Env(params=params)

Set parameter Username
Set parameter LicenseID to value 2684219
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2697514
Academic license 2697514 - for non-commercial use only - registered to c2___@dac.unicamp.br


#### 2. Gerador de Instância

##### 2.1 Função que gera uma instância, podendo ter 3 padrões:

- Padrão 1: Subconjuntos Pequenos (Alta Fragmentação)
- Padrão 2: Subconjuntos Médios (Estrutura Circular)
- Padrão 3: Subconjuntos com Tamanhos Mistos

In [4]:
# instance generation function
def generate_instance(n, pattern_type, seed=None):
    """
    Generate MAX-SC-QBF instance with different coverage patterns
    pattern_type: 1, 2, or 3 for different subset generation strategies
    """
    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)

    # Generate coefficient matrix A
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):
            A[i][j] = np.random.uniform(-10, 10)

    # Generate subsets based on pattern type
    subsets = []

    if pattern_type == 1:
        # Pattern 1: Small overlapping subsets (2-4 elements each)
        for i in range(n):
            size = np.random.randint(2, min(5, n//2 + 1))
            elements = set()
            elements.add(i)
            while len(elements) < size:
                elements.add(np.random.randint(0, n))
            subsets.append(sorted(list(elements)))

    elif pattern_type == 2:
        # Pattern 2: Medium-sized subsets with overlap (~3 elements)
        coverage_target = 3
        for i in range(n):
            size = np.random.randint(max(2, n//10), max(3, n//5 + 1))
            base = i
            elements = set()
            for j in range(size):
                elements.add((base + j) % n)
            subsets.append(sorted(list(elements)))

    else:
        # Pattern 3: Mixed sizes with some large subsets
        for i in range(n):
            prob = np.random.random()
            if prob < 0.3:  # Small subset
                size = np.random.randint(2, max(3, n//10 + 1))
            elif prob < 0.8:  # Medium subset
                size = np.random.randint(max(3, n//8), max(4, n//4 + 1))
            else:  # Large subset
                size = np.random.randint(max(4, n//3), max(5, n//2 + 1))

            elements = set([i])  # Start with element i
            candidates = list(range(n))
            candidates.remove(i)
            np.random.shuffle(candidates)
            for j in candidates[:size-1]:
                elements.add(j)
            subsets.append(sorted(list(elements)))

    # Ensure all elements are covered
    covered = set()
    for s in subsets:
        covered.update(s)

    for elem in range(n):
        if elem not in covered:
            subsets[elem % len(subsets)].append(elem)

    return A, subsets

##### 2.2 Funções para salvar e carregar instâncias

In [5]:
# save instances to dir
def save_instance(filename, n, A, subsets):
    """Save instance in the specified format"""
    with open(filename, 'w') as f:
        f.write(f"{n}\n")

        # Write subset sizes
        sizes = [len(s) for s in subsets]
        f.write(" ".join(map(str, sizes)) + "\n")

        # Write subset elements
        for subset in subsets:
            f.write(" ".join(map(str, [x+1 for x in subset])) + "\n")

        # Write upper triangular matrix
        for i in range(n):
            row = []
            for j in range(i, n):
                row.append(str(A[i][j]))
            f.write(" ".join(row) + "\n")

# load instances to dir
def load_instance(filename):
    """Load instance from file"""
    with open(filename, 'r') as f:
        lines = f.readlines()

    n = int(lines[0].strip())
    sizes = list(map(int, lines[1].strip().split()))

    subsets = []
    line_idx = 2
    for size in sizes:
        subset = list(map(int, lines[line_idx].strip().split()))
        subsets.append([x-1 for x in subset])
        line_idx += 1

    A = np.zeros((n, n))
    for i in range(n):
        values = list(map(float, lines[line_idx].strip().split()))
        for j, val in enumerate(values):
            A[i][i+j] = val
        line_idx += 1

    return n, A, subsets

#### 3. Implementação do modelo matemático

In [6]:
# Implement the mathematical formulation
def solve_max_sc_qbf(n, A, subsets, time_limit=600, env=None):
    """
    Solve MAX-SC-QBF using Gurobi
    Returns: objective value, lower bound, upper bound, gap, solving time
    """
    model = gp.Model("MAX-SC-QBF", env=env)

    # Decision variables
    x = model.addVars(n, vtype=GRB.BINARY, name="x")
    y = model.addVars(n, n, vtype=GRB.BINARY, name="y")

    # Objective function
    obj = gp.quicksum(A[i][j] * y[i,j] for i in range(n) for j in range(n))
    model.setObjective(obj, GRB.MAXIMIZE)

    # Linearization constraints
    for i in range(n):
        for j in range(n):
            model.addConstr(y[i,j] <= x[i], name=f"lin1_{i}_{j}")
            model.addConstr(y[i,j] <= x[j], name=f"lin2_{i}_{j}")
            model.addConstr(y[i,j] >= x[i] + x[j] - 1, name=f"lin3_{i}_{j}")

    # Set cover constraints
    for k in range(n):
        covering_sets = [i for i in range(n) if k in subsets[i]]
        if covering_sets:
            model.addConstr(
                gp.quicksum(x[i] for i in covering_sets) >= 1,
                name=f"cover_{k}"
            )

    # Set parameters
    model.Params.TimeLimit = time_limit
    model.Params.MIPGap = 0.0001
    model.Params.OutputFlag = 0

    # Solve
    start_time = time.time()
    model.optimize()
    solve_time = time.time() - start_time

    # Extract results
    if model.Status == GRB.OPTIMAL:
        obj_val = model.ObjVal
        obj_bound = model.ObjBound
        gap = 0.0
    elif model.Status == GRB.TIME_LIMIT:
        obj_val = model.ObjVal if model.SolCount > 0 else 0
        obj_bound = model.ObjBound
        gap = model.MIPGap * 100
    else:
        obj_val = 0
        obj_bound = 0
        gap = 100.0

    return {
        'LI': obj_val,
        'LS': obj_bound,
        'Gap': gap,
        'Time': solve_time,
        'Status': model.Status
    }

#### 4. Funções auxiliares

##### 4.1 Função que gera n instâncias

In [12]:
# Generete all instances
def generate_all_instances(output_dir="instances", n_values = [25, 50, 100, 200, 400], patterns = [1, 2, 3]):
    """Generate all instances"""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    instances = []
    for i, n in enumerate(n_values):
        for j, pattern in enumerate(patterns):
            instance_name = f"n{n}p{pattern}"
            filename = os.path.join(output_dir, f"{instance_name}.txt")

            A, subsets = generate_instance(n, pattern, seed=42+i*10+j)
            save_instance(filename, n, A, subsets)
            instances.append((instance_name, n, pattern))
            print(f"Generated instance {instance_name} (n={n}, pattern={pattern})")

    return instances

##### 4.2 Função que roda os experimentos

In [8]:
# Run experiments and collect results
def run_experiments(instance_dir="instances", time_limit=600, env=None):
    """Run all experiments and collect results"""
    results = []

    # Get all instance files
    instance_files = sorted([f for f in os.listdir(instance_dir) if f.endswith('.txt')])

    for instance_file in instance_files:
        instance_name = instance_file.replace('.txt', '')
        filepath = os.path.join(instance_dir, instance_file)

        print(f"\nSolving {instance_name}...")

        # Load instance
        n, A, subsets = load_instance(filepath)

        # Solve
        result = solve_max_sc_qbf(n, A, subsets, time_limit, env)

        # Add instance info
        result['Instance'] = instance_name
        result['n'] = n

        results.append(result)

        print(f"  LI: {result['LI']:.2f}")
        print(f"  LS: {result['LS']:.2f}")
        print(f"  Gap: {result['Gap']:.2f}%")
        print(f"  Time: {result['Time']:.2f}s")

    # Create DataFrame
    df = pd.DataFrame(results)
    df = df[['Instance', 'n', 'LS', 'LI', 'Gap', 'Time']]

    return df

#### 5. Rotina de execução

In [ ]:
# Generate instances
instances = generate_all_instances()

# Run experiments
results_df = run_experiments(time_limit=600, env=env)

# Display results
print("\n" + "="*60)
print("RESULTS")
print("="*60)
print(results_df.to_string(index=False))

# Save results to CSV
results_df.to_csv("results.csv", index=False)

Generated instance n25p1 (n=25, pattern=1)
Generated instance n25p2 (n=25, pattern=2)
Generated instance n25p3 (n=25, pattern=3)
Generated instance n50p1 (n=50, pattern=1)
Generated instance n50p2 (n=50, pattern=2)
Generated instance n50p3 (n=50, pattern=3)
Generated instance n100p1 (n=100, pattern=1)
Generated instance n100p2 (n=100, pattern=2)
Generated instance n100p3 (n=100, pattern=3)
Generated instance n200p1 (n=200, pattern=1)
Generated instance n200p2 (n=200, pattern=2)
Generated instance n200p3 (n=200, pattern=3)
Generated instance n400p1 (n=400, pattern=1)
Generated instance n400p2 (n=400, pattern=2)
Generated instance n400p3 (n=400, pattern=3)

Solving n100p1...
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.0001
  LI: 0.00
  LS: 0.00
  Gap: 100.00%
  Time: 0.03s

Solving n100p2...
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.0001

Interrupt request received
